In [ ]:
# faceMask and faceMask2 hDDM face ratings analysis script: mask analyses of false negatives (failing to ID correct emotion)
# 22/03/21

# note: to be run within python 3.5 environment named "hddm" (from console: 'source activate hddm')
# confirm jupyter notebook is launching from "hddm" environment (see top-right: 'conda env:hddm')

# set up
import numpy as np
print(np.__version__) # should be 1.11.3

import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from patsy import dmatrix

import hddm
print(hddm.__version__) # should be 0.6.0

In [ ]:
# load hDDM-processed data from CSV file into a NumPy structured array
data1 = hddm.load_csv('faceMask_hddm_proc.csv')
                     
# flip 'no' response RTs to be negative
data1 = hddm.utils.flip_errors(data1)

# check dataset
data1.head(10)

In [ ]:
# plot RTs for each participant
fig = plt.figure()
ax = fig.add_subplot(111, xlabel='RT', ylabel='count', title='RT distributions')
for i, subj_data in data1.groupby('subj_idx'):
    subj_data.rt.hist(bins=20, histtype='step', ax=ax)

In [ ]:
# subset dataset by emotion rating type (disgusted) and facial expression (angry)
dd1_data = data1.loc[(data1['emotionRating'] == 'disgusted') & (data1['expression'] == 'disgusted')]
dd1_data.head(10)

In [ ]:
# drop any subjects missing columns in design matrix (otherwise produces error)
dd1_data = dd1_data[(dd1_data.subj_idx != 'ygpcur698h4dcw5') & (dd1_data.subj_idx != 'xb95r55zxl5xgqr')]

In [ ]:
# define model
m_dd1 = hddm.HDDMRegressor(dd1_data, "v ~ C(mask)", bias=True, p_outlier=0.05)

In [ ]:
# run model
m_dd1.sample(5000, burn=200, dbname='traces.db', db='pickle')
m_dd1.save('hddm_dd1')

In [ ]:
m_dd1.print_stats()
m_dd1.plot_posteriors()

In [ ]:
# save model stats
dd1_stats = m_dd1.gen_stats()
print(dd1_stats)
dd1_stats.to_csv('dd1_5000/faceMask_hddm_drift_maskFalseNegatives_dd1_5000.csv', index=True)

# save model posterior plots
m_dd1.plot_posteriors(path='dd1_5000/_posteriors', save=True)

In [ ]:
# plot model posteriors by mask
dd1_v_none, dd1_v_lower, dd1_v_upper = m_dd1.nodes_db.node[["v_Intercept", "v_C(mask)[T.lower]", "v_C(mask)[T.upper]"]]
hddm.analyze.plot_posterior_nodes([dd1_v_none, dd1_v_lower, dd1_v_upper])
plt.savefig('dd1_5000/faceMask_hddm_drift_maskFalsePositives_dd1_5000_v_mask.pdf')

In [ ]:
## estimate probabilities that mask coefficient posteriors differ from 0
# note that comparison condition coefficients (i.e. lower/upper) are *relative to* baseline condition (i.e. no mask)
# for lower/upper, =0 means no change from baseline, <0 means less than baseline, and >0 means greater than baseline
# for no mask, =0 means null drift, <0 means negative drift, and >0 means positive drift
print("P(dd1_v_none < 0) = ", (dd1_v_none.trace() < 0).mean())
print("P(dd1_v_lower > 0) = ", (dd1_v_lower.trace() > 0).mean())
print("P(dd1_v_upper > 0) = ", (dd1_v_upper.trace() > 0).mean())
# estimate probability that lower and upper mask coefficient posteriors differ
print("P(dd1_v_lower > dd1_v_upper) = ", (dd1_v_lower.trace() > dd1_v_upper.trace()).mean())

In [ ]:
# load hDDM-processed data from CSV file into a NumPy structured array
data2 = hddm.load_csv('faceMask2_hddm_proc.csv')
                     
# flip 'no' response RTs to be negative
data2 = hddm.utils.flip_errors(data2)

# check dataset
data2.head(10)

In [ ]:
# plot RTs for each participant
fig = plt.figure()
ax = fig.add_subplot(111, xlabel='RT', ylabel='count', title='RT distributions')
for i, subj_data in data2.groupby('subj_idx'):
    subj_data.rt.hist(bins=20, histtype='step', ax=ax)

In [ ]:
# subset dataset by emotion rating type (disgusted) and facial expression (angry)
dd2_data = data2.loc[(data2['emotionRating'] == 'disgusted') & (data2['expression'] == 'disgusted')]
dd2_data.head(10)

In [ ]:
# drop any subjects missing columns in design matrix (otherwise produces error)
dd2_data = dd2_data[(dd2_data.subj_idx != 'hdn2l3df216jykn')]

In [ ]:
# define model
m_dd2 = hddm.HDDMRegressor(dd2_data, "v ~ C(mask)", bias=True, p_outlier=0.05)

In [ ]:
# run model
m_dd2.sample(5000, burn=200, dbname='traces.db', db='pickle')
m_dd2.print_stats()
m_dd2.plot_posteriors()

In [ ]:
# save model stats
dd2_stats = m_dd2.gen_stats()
print(dd2_stats)
dd2_stats.to_csv('dd2_5000/faceMask2_hddm_drift_maskFalseNegatives_dd2_5000.csv', index=True)

# save model posterior plots
m_dd2.plot_posteriors(path='dd2_5000/_posteriors', save=True)

In [ ]:
# plot model posteriors by mask
dd2_v_none, dd2_v_lower, dd2_v_upper = m_dd2.nodes_db.node[["v_Intercept", "v_C(mask)[T.lower]", "v_C(mask)[T.upper]"]]
hddm.analyze.plot_posterior_nodes([dd2_v_none, dd2_v_lower, dd2_v_upper])
plt.savefig('dd2_5000/faceMask2_hddm_drift_maskFalsePositives_dd2_5000_v_mask.pdf')

In [ ]:
## estimate probabilities that mask coefficient posteriors differ from 0
# note that comparison condition coefficients (i.e. lower/upper) are *relative to* baseline condition (i.e. no mask)
# for lower/upper, =0 means no change from baseline, <0 means less than baseline, and >0 means greater than baseline
# for no mask, =0 means null drift, <0 means negative drift, and >0 means positive drift
print("P(dd2_v_none < 0) = ", (dd2_v_none.trace() < 0).mean())
print("P(dd2_v_lower > 0) = ", (dd2_v_lower.trace() > 0).mean())
print("P(dd2_v_upper > 0) = ", (dd2_v_upper.trace() > 0).mean())
# estimate probability that lower and upper mask coefficient posteriors differ
print("P(dd2_v_lower > dd2_v_upper) = ", (dd2_v_lower.trace() > dd2_v_upper.trace()).mean())

In [ ]:
## estimate probabilities that mask coefficient posteriors differ from each other (faceMask 1 vs. faceMask2)
# note that comparison condition coefficients (i.e. lower/upper) are *relative to* baseline condition (i.e. no mask)
# for lower/upper, =0 means no change from baseline, <0 means less than baseline, and >0 means greater than baseline
# for no mask, =0 means null drift, <0 means negative drift, and >0 means positive drift
print("P(dd1_v_none < dd2_v_none) = ", (dd1_v_none.trace() < dd2_v_none.trace()).mean())
print("P(dd1_v_lower < dd2_v_lower) = ", (dd1_v_lower.trace() < dd2_v_lower.trace()).mean())
print("P(dd1_v_upper < dd2_v_upper) = ", (dd1_v_upper.trace() < dd2_v_upper.trace()).mean())


In [ ]:
## adding together the relative values:
dd1_v_none_plus_lower = dd1_v_none.trace() + dd1_v_lower.trace()
dd1_v_none_plus_upper = dd1_v_none.trace() + dd1_v_upper.trace()

dd2_v_none_plus_lower = dd2_v_none.trace() + dd2_v_lower.trace()
dd2_v_none_plus_upper = dd2_v_none.trace() + dd2_v_upper.trace()

In [ ]:
print("P(dd1_v_none_plus_lower < dd2_v_none_plus_lower) = ", (dd1_v_none_plus_lower < dd2_v_none_plus_lower).mean())
print("P(dd1_v_none_plus_upper < dd2_v_none_plus_upper) = ", (dd1_v_none_plus_upper < dd2_v_none_plus_upper).mean())